In [2]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('partitions')
sc = SparkContext(conf = conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/05 05:05:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 사용자 지정 파티션 생성하기

In [4]:
pairs = sc.parallelize([
    1,2,3,4,2,4,1
]).map(lambda x: (x,x))

pairs.collect()

[(1, 1), (2, 2), (3, 3), (4, 4), (2, 2), (4, 4), (1, 1)]

In [5]:
# key를 2로 나눈 나머지에 따라서 파티션 나눈 듯.
# hash algorithm에 따라서 나뉜,...
pairs.partitionBy(2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [12]:
# 개발자가 직접 파티션 배치 함수를 만들 수 있다.
pairs.partitionBy(2, lambda x: x%2).glom().collect()

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]

In [8]:
pairs.partitionBy(3, lambda x: x%3).glom().collect()

[[(3, 3)], [(1, 1), (4, 4), (4, 4), (1, 1)], [(2, 2), (2, 2)]]

파티션을 만든 후에 `persist()`를 하지 않으면 다음 연산에 불릴 때마다 파티션을 만드는 코드가 계속 반복된다.
- 셔플링이 반복적으로 계속 일어난다.
- persist 안하면 기존의 파티션을 무시하게 됨....
- 따라서 파티션 만들고 persist 하는게 좋다.

In [13]:
pairs.partitionBy(2, lambda x: x%2).persist().glom().collect()
# 결과물은 똑같지만, 이 코드 다시 실행해도 새로운 파티션을 만들지 않음.

[[(2, 2), (4, 4), (2, 2), (4, 4)], [(1, 1), (3, 3), (1, 1)]]